In [18]:
import csv
import pickle
import random
import networkx as nx

In [2]:
import numpy as np
import pandas as pd

In [62]:
import csv

In [3]:
train = pd.read_csv('2hop_crawl.csv')

In [6]:
len(train[train['Connected'] == 1])

34540

In [14]:
edge_list = list(zip([x for x in train[train['Connected'] == 1]['Souce']], [x for x in train[train['Connected'] == 1]['Sink']]))

In [15]:
edge_list[:5]

[(1933312, 3025121),
 (1933312, 3392171),
 (1933312, 3542411),
 (1933312, 38814),
 (1933312, 1348407)]

In [19]:
G = nx.Graph()
G.add_edges_from(edge_list)

In [21]:
print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 29857
Number of edges: 34517
Average degree:   2.3122


In [40]:
J = nx.DiGraph()
J.add_edges_from(edge_list)

In [41]:
page_rank = nx.pagerank_scipy(J)

In [25]:
mixed_pairs = edge_list

In [26]:
jc = list(networkx.jaccard_coefficient(G, mixed_pairs))

In [27]:
ra = list(networkx.resource_allocation_index(G, mixed_pairs))

In [28]:
aa = list(networkx.adamic_adar_index(G, mixed_pairs))
cn = [(e[0], e[1], len(list(networkx.common_neighbors(G,e[0], e[1])))) for e in mixed_pairs]

In [57]:
pa = list(networkx.preferential_attachment(G, mixed_pairs))

In [35]:
#wic = list(networkx.within_inter_cluster(G, mixed_pairs))

In [36]:
string_pairs = mixed_pairs

In [43]:
page_rank_source = []
page_rank_sink = []

 
for (x,y) in string_pairs:
    page_rank_source.append(page_rank[x])
    page_rank_sink.append(page_rank[y])

In [44]:
hub_score, authority_score = nx.hits(J)

In [59]:
hub_source = []
hub_sink = []
authority_source = []
authority_sink = []

for (x,y) in string_pairs:
    hub_source.append(hub_score[x])
    hub_sink.append(hub_score[y])    
    authority_source.append(authority_score[x])
    authority_sink.append(authority_score[y])

In [45]:
out_degree_centrality = nx.out_degree_centrality(J)
in_degree_centrality = nx.in_degree_centrality(J)

In [46]:
source_out_degree = []
source_in_degree = []
sink_out_degree = []
sink_in_degree = []

for (x,y) in string_pairs:
    source_out_degree.append(out_degree_centrality[x])
    sink_out_degree.append(out_degree_centrality[y])    
    source_in_degree.append(in_degree_centrality[x])
    sink_in_degree.append(in_degree_centrality[y])

In [51]:
sp = []
for (x,y) in string_pairs:
    try:
        sp.append(nx.shortest_path_length(J, x, y))
    except:
        sp.append(2) # find the max and add one

In [49]:
max(sp)

1

In [108]:
train_list = list(zip([x for x in train['Souce']], [x for x in train['Sink']]))

In [113]:
label = [x for x in train['Connected']]

In [114]:
jc, ra, aa, cn, pa, page_rank_source, page_rank_sink, hub_source, hub_sink, authority_source, authority_sink, out_degree_centrality, in_degree_centrality, source_out_degree, sink_out_degree, source_in_degree, sink_in_degree, sp = generate_features(G, J, train_list)

In [115]:
len(jc)

69080

In [118]:
write_data = [("Source", "Sink", "Connected", "Jaccard", "Resource_alloc", "Adamic_adar", "Preferential_attachment", "Common_Neighbours", "Shortest_Path_Length","Page_source", "Page_sink","Hub_source","Hub_sink","Authority_Source","Authority_Sink","Source_Out","Source_In","Sink_out","Sink_in")]
for i,pair in enumerate(train_list):
    entry = (pair[0], pair[1], label[i], jc[i][2], ra[i][2], aa[i][2], pa[i][2], cn[i][2], sp[i], page_rank_source[i], page_rank_sink[i], hub_source[i], hub_sink[i], authority_source[i], authority_sink[i], source_out_degree[i], source_in_degree[i], sink_out_degree[i], sink_in_degree[i])
    write_data.append(entry)

In [119]:
with open("./trainData.csv", 'w', newline="") as f:
    writer = csv.writer(f)
    writer.writerows(write_data)

In [83]:
test_list = []
with open("./test-public.txt", 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    
    for line in reader:
        test_list.append((line[1], line[2]))

In [84]:
test_list = test_list[1:]

In [92]:
test_list = [(int(x[0]), int(x[1])) for x in test_list]

In [85]:
test_list[:5]

[('2184483', '1300190'),
 ('3151356', '1452193'),
 ('1579396', '193159'),
 ('1406432', '2481036'),
 ('2389638', '593017')]

In [78]:
2184483

[(1933312, 3025121),
 (1933312, 3392171),
 (1933312, 3542411),
 (1933312, 38814),
 (1933312, 1348407)]

In [96]:
len(test_list)

2000

In [93]:
jc, ra, aa, cn, pa, page_rank_source, page_rank_sink, hub_source, hub_sink, authority_source, authority_sink, out_degree_centrality, in_degree_centrality, source_out_degree, sink_out_degree, source_in_degree, sink_in_degree, sp = generate_features(G, J, test_list)

In [99]:
write_data = [("Source", "Sink", "Jaccard", "Resource_alloc", "Adamic_adar", "Preferential_attachment", "Common_Neighbours", "Shortest_Path_Length","Page_source", "Page_sink","Hub_source","Hub_sink","Authority_Source","Authority_Sink","Source_Out","Source_In","Sink_out","Sink_in")]
for i,pair in enumerate(test_list):
    entry = (pair[0], pair[1], jc[i][2], ra[i][2], aa[i][2], pa[i][2], cn[i][2], sp[i], page_rank_source[i], page_rank_sink[i], hub_source[i], hub_sink[i], authority_source[i], authority_sink[i], source_out_degree[i], source_in_degree[i], sink_out_degree[i], sink_in_degree[i])
    write_data.append(entry)
# jc, ra, aa, cn, pa, page_rank_source, page_rank_sink, hub_source, hub_sink, authority_source, authority_sink, out_degree_centrality, in_degree_centrality, source_out_degree, sink_out_degree, source_in_degree, sink_in_degree, sp

In [100]:
with open("./testData.csv", 'w', newline="") as f:
    writer = csv.writer(f)
    writer.writerows(write_data)

In [79]:
def generate_features(G, J, mixed_pairs):
    jc = list(nx.jaccard_coefficient(G, mixed_pairs))
    ra = list(nx.resource_allocation_index(G, mixed_pairs))
    aa = list(nx.adamic_adar_index(G, mixed_pairs))
    cn = [(e[0], e[1], len(list(nx.common_neighbors(G,e[0], e[1])))) for e in mixed_pairs]
    pa = list(nx.preferential_attachment(G, mixed_pairs))
    
    page_rank_source = []
    page_rank_sink = []

 
    for (x,y) in mixed_pairs:
        page_rank_source.append(page_rank[x])
        page_rank_sink.append(page_rank[y])
    
    hub_score, authority_score = nx.hits(J)
    
    hub_source = []
    hub_sink = []
    authority_source = []
    authority_sink = []

    for (x,y) in mixed_pairs:
        hub_source.append(hub_score[x])
        hub_sink.append(hub_score[y])    
        authority_source.append(authority_score[x])
        authority_sink.append(authority_score[y])
        
    out_degree_centrality = nx.out_degree_centrality(J)
    in_degree_centrality = nx.in_degree_centrality(J)
    
    source_out_degree = []
    source_in_degree = []
    sink_out_degree = []
    sink_in_degree = []

    for (x,y) in mixed_pairs:
        source_out_degree.append(out_degree_centrality[x])
        sink_out_degree.append(out_degree_centrality[y])    
        source_in_degree.append(in_degree_centrality[x])
        sink_in_degree.append(in_degree_centrality[y])
        
    sp = []
    for (x,y) in mixed_pairs:
        try:
            sp.append(nx.shortest_path_length(J, x, y))
        except:
            sp.append(2) # find the max and add one
            
    return jc, ra, aa, cn, pa, page_rank_source, page_rank_sink, hub_source, hub_sink, authority_source, authority_sink, out_degree_centrality, in_degree_centrality, source_out_degree, sink_out_degree, source_in_degree, sink_in_degree, sp 


In [70]:
def t():
    return 1

SyntaxError: invalid syntax (<ipython-input-70-48d86c1d9364>, line 2)

In [68]:
t()

(1, 2)